# Praca domowa 2 - Mikołaj Jakubowski

### biblioteki

In [10]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

### wczytanie danych i krótkie rozeznanie

In [53]:
data = pd.read_csv('https://www.dropbox.com/s/360xhh2d9lnaek3/allegro-api-transactions.csv?dl=1')

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420020 entries, 0 to 420019
Data columns (total 14 columns):
lp                        420020 non-null int64
date                      420020 non-null object
item_id                   420020 non-null int64
categories                420020 non-null object
pay_option_on_delivery    420020 non-null int64
pay_option_transfer       420020 non-null int64
seller                    420020 non-null object
price                     420020 non-null float64
it_is_allegro_standard    420020 non-null int64
it_quantity               420020 non-null int64
it_is_brand_zone          420020 non-null int64
it_seller_rating          420020 non-null int64
it_location               420020 non-null object
main_category             420020 non-null object
dtypes: float64(1), int64(8), object(5)
memory usage: 44.9+ MB


In [13]:
data.it_location.unique().size

10056

### target encoding dla 'it_location'

In [14]:
import category_encoders as ce

In [15]:
te=ce.target_encoder.TargetEncoder(data.it_location)
encoded=te.fit_transform(data.it_location, data.price)
encoded

,it_location
0,85.423398
1,85.423398
2,61.990914
3,35.433365
4,117.191956
5,8.394330
6,8.394330
7,61.893838
8,43.808448
9,85.423398


In [16]:
# ten sam wynik da:
targ_encoded2=data['it_location'].map(data.groupby('it_location')['price'].mean())
targ_encoded2.head()

0     85.423398
1     85.423398
2     61.990914
3     35.433365
4    117.191956
Name: it_location, dtype: float64

Co ciekawe bardzo prosta funkcja, bazująca na prostych metodach pandasa zamknięta w dość skomplikowane 'opakowanie'. Ale może jest to bonus do bardziej skomplikowanych encoderów w tym pakiecie.

Przewaga target encodingu nad one-hot'em jest głównie taka, że one-hot potrzebowałby tutaj 10 tysięcy dodatkowych kolumn, co może być nieco problematyczne dla naszej pamięci. Jednak wadą może być to, że jakiekolwiek uzależnianie zmiennej opisującej, od tej docelowej może nam nieco przysłaniać poprzednie informacje.

## kodowanie kolumny 'main_category'

In [17]:
data.main_category.unique().size

27

nieco mniej więc one-hot encoding może mieć już jakiś sens

### One-hot

In [18]:
ohe = ce.OneHotEncoder(cols=['main_category'])
df_onehot = ohe.fit_transform(data)
df_onehot.head()

,lp,date,item_id,categories,pay_option_on_delivery,pay_option_transfer,seller,price,it_is_allegro_standard,it_quantity,...,main_category_18,main_category_19,main_category_20,main_category_21,main_category_22,main_category_23,main_category_24,main_category_25,main_category_26,main_category_27
0,0,2016-04-03 21:21:08,4753602474,"['Komputery', 'Dyski i napędy', 'Nośniki', 'No...",1,1,radzioch666,59.99,1,997,...,0,0,0,0,0,0,0,0,0,0
1,1,2016-04-03 15:35:26,4773181874,"['Odzież, Obuwie, Dodatki', 'Bielizna damska',...",1,1,InwestycjeNET,4.90,1,9288,...,0,0,0,0,0,0,0,0,0,0
2,2,2016-04-03 14:14:31,4781627074,"['Dom i Ogród', 'Budownictwo i Akcesoria', 'Śc...",1,1,otostyl_com,109.90,1,895,...,0,0,0,0,0,0,0,0,0,0
3,3,2016-04-03 19:55:44,4783971474,"['Książki i Komiksy', 'Poradniki i albumy', 'Z...",1,1,Matfel1,18.50,0,971,...,0,0,0,0,0,0,0,0,0,0
4,4,2016-04-03 18:05:54,4787908274,"['Odzież, Obuwie, Dodatki', 'Ślub i wesele', '...",1,1,PPHU_RICO,19.90,1,950,...,0,0,0,0,0,0,0,0,0,0


### Leave One Out

Metoda bardzo podobna do 'target encoding'. Różnią się jedynie tym, że w momencie liczenia średniej pominięty zostaje aktualnie sprawdzany wiersz. Ma to zapobiec wpływie outlierów.

In [19]:
loo = ce.LeaveOneOutEncoder(cols=['main_category'])
df_loo = loo.fit_transform(data, data.price)
df_loo.head()

,lp,date,item_id,categories,pay_option_on_delivery,pay_option_transfer,seller,price,it_is_allegro_standard,it_quantity,it_is_brand_zone,it_seller_rating,it_location,main_category
0,0,2016-04-03 21:21:08,4753602474,"['Komputery', 'Dyski i napędy', 'Nośniki', 'No...",1,1,radzioch666,59.99,1,997,0,50177,Warszawa,121.814331
1,1,2016-04-03 15:35:26,4773181874,"['Odzież, Obuwie, Dodatki', 'Bielizna damska',...",1,1,InwestycjeNET,4.90,1,9288,0,12428,Warszawa,75.859357
2,2,2016-04-03 14:14:31,4781627074,"['Dom i Ogród', 'Budownictwo i Akcesoria', 'Śc...",1,1,otostyl_com,109.90,1,895,0,7389,Leszno,72.434342
3,3,2016-04-03 19:55:44,4783971474,"['Książki i Komiksy', 'Poradniki i albumy', 'Z...",1,1,Matfel1,18.50,0,971,0,15006,Wola Krzysztoporska,25.028061
4,4,2016-04-03 18:05:54,4787908274,"['Odzież, Obuwie, Dodatki', 'Ślub i wesele', '...",1,1,PPHU_RICO,19.90,1,950,0,32975,BIAŁYSTOK,75.859080


### Hashing

In [40]:
hsh = ce.HashingEncoder(cols='main_category')
df_hsh = hsh.fit_transform(data)
df_hsh.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,lp,date,...,categories,pay_option_on_delivery,pay_option_transfer,seller,price,it_is_allegro_standard,it_quantity,it_is_brand_zone,it_seller_rating,it_location
0,0,0,0,0,0,0,1,0,0,2016-04-03 21:21:08,...,"['Komputery', 'Dyski i napędy', 'Nośniki', 'No...",1,1,radzioch666,59.99,1,997,0,50177,Warszawa
1,0,0,1,0,0,0,0,0,1,2016-04-03 15:35:26,...,"['Odzież, Obuwie, Dodatki', 'Bielizna damska',...",1,1,InwestycjeNET,4.90,1,9288,0,12428,Warszawa
2,0,0,0,0,0,0,1,0,2,2016-04-03 14:14:31,...,"['Dom i Ogród', 'Budownictwo i Akcesoria', 'Śc...",1,1,otostyl_com,109.90,1,895,0,7389,Leszno
3,0,0,1,0,0,0,0,0,3,2016-04-03 19:55:44,...,"['Książki i Komiksy', 'Poradniki i albumy', 'Z...",1,1,Matfel1,18.50,0,971,0,15006,Wola Krzysztoporska
4,0,0,1,0,0,0,0,0,4,2016-04-03 18:05:54,...,"['Odzież, Obuwie, Dodatki', 'Ślub i wesele', '...",1,1,PPHU_RICO,19.90,1,950,0,32975,BIAŁYSTOK


Działa nieco mniej obszernie(zabiera mniej pamięci) niż one-hot. Pytanie czy jest to skuteczne kodowanie. Z pewnością działa o wiele wolniej niż reszta dotychczasowych.

## Uzupełnianie braków

In [185]:
data_old = data
data = data[['price', 'it_seller_rating', 'it_quantity']]

### uzupełnianie zmiennej 'it_seller_rating'

usuwanie:

In [186]:
n=len(data)
m=int(n*0.1)
to_remove=np.random.choice(n, m)
data.loc[to_remove,'it_seller_rating'] = np.nan

Multivariate feature imputation

In [187]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imp = IterativeImputer(max_iter=10, random_state=0)
#imp.fit(data)
data[['it_seller_rating']]=imp.fit_transform(data)[:,1].astype(int)

Błąd:

In [188]:
from sklearn.metrics import mean_squared_error
mean_squared_error(data_old.it_seller_rating, data.it_seller_rating)

129214445.79734774

Czy tu wszystko jest ok? strasznie duży ten błąd

In [189]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420020 entries, 0 to 420019
Data columns (total 3 columns):
price               420020 non-null float64
it_seller_rating    420020 non-null int64
it_quantity         420020 non-null int64
dtypes: float64(1), int64(2)
memory usage: 9.6 MB


In [190]:
data

,price,it_seller_rating,it_quantity
0,59.99,50177,997
1,4.90,12428,9288
2,109.90,7389,895
3,18.50,20020,971
4,19.90,20017,950
5,10.99,39371,742
6,10.99,39371,742
7,28.99,4287,886
8,12.00,5403,49
9,45.00,15561,4


In [191]:
data_old[['price', 'it_seller_rating', 'it_quantity']]

,price,it_seller_rating,it_quantity
0,59.99,50177,997
1,4.90,12428,9288
2,109.90,7389,895
3,18.50,15006,971
4,19.90,32975,950
5,10.99,39371,742
6,10.99,39371,742
7,28.99,4287,886
8,12.00,5403,49
9,45.00,15561,4


##### Eksperyment powtórzony 10 razy

In [192]:
data=data_old[['price', 'it_seller_rating', 'it_quantity']]

In [193]:
for i in range(10):
    to_remove=np.random.choice(n, m)
    data.loc[to_remove,'it_seller_rating'] = np.nan
    data[['it_seller_rating']]=imp.fit_transform(data)[:,1].astype(int)

mean_squared_error(data_old.it_seller_rating, data.it_seller_rating)

846663942.8412766

### Ten sam eksperyment ale dla dwóch usunietych kolumn:

In [197]:
data=data_old[['price', 'it_seller_rating', 'it_quantity']]

In [198]:
for i in range(10):
    to_remove=np.random.choice(n, m)
    to_remove2=np.random.choice(n, m)
    data.loc[to_remove,'it_seller_rating'] = np.nan
    data.loc[to_remove2, 'it_quantity'] = np.nan
    data[['it_seller_rating']]=imp.fit_transform(data)[:,1].astype(int)
    data[['it_seller_rating']]=imp.fit_transform(data)[:,2].astype(int)

ValueError: shape mismatch: value array of shape (420020,) could not be broadcast to indexing result of shape (1,420020)

In [154]:
mean_squared_error(data_old.it_seller_rating, data.it_seller_rating)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [168]:
    to_remove=np.random.choice(n, m)
    to_remove2=np.random.choice(n, m)
    data.loc[to_remove,'it_seller_rating'] = np.nan
    data.loc[to_remove2, 'it_quantity'] = np.nan
    X=imp.fit_transform(data)

In [199]:
imp.fit_transform(data)

array([[5.9990e+01, 5.0177e+04, 9.9700e+02],
       [4.9000e+00, 1.2428e+04, 9.2880e+03],
       [1.0990e+02, 7.3890e+03, 8.9500e+02],
       ...,
       [5.9900e+00, 9.8300e+02, 4.7000e+02],
       [2.0000e+02, 1.6300e+02, 0.0000e+00],
       [5.0000e+02, 2.6500e+02, 0.0000e+00]])

Niestety nie potrafię ogarnąć tych błędów pod koniec.